In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first') ## setting image format -- 
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [4]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [5]:
def triplet_loss(y_true, y_pred, alpha = 0.2):

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(basic_loss,0)
    ### END CODE HERE ###
    
    return loss

In [6]:
## Testing triplet loss function
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 350.02716


In [7]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [9]:
database = {}
database["danielle"] = img_to_encoding1("genuine/NFI-00101001.png", FRmodel)
database["younes"] = img_to_encoding1("genuine/NFI-00201002.png", FRmodel)
database["tian"] = img_to_encoding1("genuine/NFI-00301003.png", FRmodel)
database["andrew"] = img_to_encoding1("genuine/NFI-00401004.png", FRmodel)
database["kian"] = img_to_encoding1("genuine/NFI-00501005.png", FRmodel)
database["dan"] = img_to_encoding1("genuine/NFI-00601006.png", FRmodel)
database["sebastiano"] = img_to_encoding1("genuine/NFI-00701007.png", FRmodel)
database["bertrand"] = img_to_encoding1("genuine/NFI-00801008.png", FRmodel)
database["kevin"] = img_to_encoding1("genuine/NFI-00901009.png", FRmodel)
# database["felix"] = img_to_encoding1("genuine/NFI-0010010010.png", FRmodel)
# database["benoit"] = img_to_encoding1("genuine/NFI-0011010011.png", FRmodel)
# database["arnaud"] = img_to_encoding1("genuine/NFI-0012010012.png", FRmodel)

In [22]:
image_path = "genuine/NFI-00101001.png"
# image_path1 = "images/andrew.jpg"

In [23]:
img3 = cv2.imread(image_path,1)

In [24]:
cv2.imshow(img,'dc')
cv2.waitKey(0)

NameError: name 'img' is not defined

In [25]:
img3.shape

(735, 1460, 3)

In [ ]:
os.path.isfile("genuine/NFI-00101001.png")

In [ ]:
type(image_path)

In [ ]:
dim = (96,96)
img3 = cv2.resize(img3,dim, interpolation = cv2.INTER_AREA)

In [ ]:
img3.shape

In [8]:
def img_to_encoding1(image_path, model):
    img1 = cv2.imread(image_path, 1)
    dim = (96,96)
    img1 = cv2.resize(img1,dim, interpolation = cv2.INTER_AREA)
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [12]:
img_to_encoding1(image_path, FRmodel)

array([[ 0.11263989,  0.01372648, -0.07991041, -0.07634889,  0.14969282,
         0.11108683,  0.10481853,  0.06506968, -0.05160766, -0.05739953,
        -0.01849299,  0.02953503,  0.08016592,  0.05512313,  0.06567927,
        -0.11784229,  0.05589263, -0.10627813, -0.07336158,  0.18701214,
         0.00916011, -0.00506025,  0.08229699,  0.03794179, -0.04990904,
        -0.07687821, -0.07512034, -0.05823411,  0.03265143, -0.02395365,
         0.00676709, -0.08564734, -0.05395222,  0.04196445, -0.03725971,
         0.08200329, -0.01395269,  0.19711958, -0.15951617, -0.02356524,
         0.0414885 ,  0.1204173 , -0.14155732,  0.02384118, -0.19512586,
         0.1286311 ,  0.02088078,  0.06298035, -0.04154909,  0.10259908,
        -0.12373152,  0.03717515, -0.08484292,  0.00301059,  0.11761731,
        -0.0493396 , -0.09768467,  0.1834165 , -0.03507635, -0.0665992 ,
        -0.09797253,  0.04131002,  0.15542704, -0.14906266,  0.06773183,
        -0.06394616,  0.06270637,  0.16180785, -0.0

In [10]:

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    ### START CODE HERE ###
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding1(image_path,model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding - database[identity])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    ### END CODE HERE ###
        
    return dist, door_open

In [12]:
verify("forged/NFI-00303002.png", "younes", database, FRmodel)

It's younes, welcome home!


(0.46179956, True)

In [15]:
img_to_encoding1("genuine/NFI-00102001.png", FRmodel)

array([[ 0.11420546,  0.01982441, -0.0712276 , -0.06899657,  0.1215038 ,
         0.10826763,  0.07529756,  0.06460126, -0.07455455, -0.07774971,
        -0.03403208,  0.05708389,  0.07577948,  0.09824125,  0.02964239,
        -0.11762753,  0.11616983, -0.09799372, -0.05768457,  0.18233794,
        -0.00244365,  0.03090169,  0.0842227 ,  0.074253  , -0.06741625,
        -0.07597209, -0.00634808,  0.00791552,  0.0046156 ,  0.00217831,
        -0.04967028, -0.09892054, -0.0170587 ,  0.07125066, -0.07000621,
         0.0572086 ,  0.01056791,  0.18884295, -0.2117404 , -0.01884297,
         0.01124215,  0.12979622, -0.07193897,  0.01186079, -0.1839701 ,
         0.11278079,  0.034254  ,  0.03981133, -0.05078958,  0.14881384,
        -0.02509809,  0.02397533, -0.07943674,  0.02678783,  0.1649583 ,
        -0.03224923, -0.09996611,  0.17062615,  0.01702928, -0.09687789,
        -0.0908179 ,  0.04814992,  0.12390123, -0.16496748,  0.06232051,
        -0.0716489 ,  0.04887782,  0.13043144, -0.0

In [17]:
img_to_encoding1("genuine/NFI-00103001.png", FRmodel)

array([[ 0.11717176,  0.04423836, -0.11427275, -0.03848497,  0.14286608,
         0.05382209,  0.06395762,  0.00444713,  0.00554071, -0.0871586 ,
         0.00803049,  0.05278345,  0.0861866 ,  0.09689719, -0.00292422,
        -0.08748735,  0.10283022, -0.10693506, -0.11507187,  0.20999338,
         0.02240222, -0.00395892,  0.07314583,  0.0421036 , -0.02757388,
        -0.07125108, -0.05095497, -0.01434501,  0.0224327 ,  0.03930177,
        -0.00660794, -0.01289282, -0.01572733,  0.04408295, -0.07684384,
         0.07413968,  0.00299342,  0.24544401, -0.14409228, -0.01436194,
         0.00410425,  0.13664557, -0.11182234, -0.00612561, -0.15001889,
         0.11832305,  0.01601231,  0.0328695 ,  0.01195212,  0.09618542,
        -0.06390809,  0.10124411, -0.07943651,  0.01180394,  0.127187  ,
        -0.07303889, -0.0930092 ,  0.13251705, -0.01785541, -0.09231742,
        -0.09570184, -0.0072075 ,  0.12259635, -0.13481224,  0.00625706,
        -0.0890718 ,  0.08139711,  0.19156003, -0.0

In [18]:
a = img_to_encoding1("genuine/NFI-00102001.png", FRmodel)

In [19]:
b = img_to_encoding1("genuine/NFI-00103001.png", FRmodel)

In [23]:
a1 = np.array([[1,2]])
b1 = np.array([[1,2]])

In [26]:
(a+b)/2

array([[ 1.15688615e-01,  3.20313871e-02, -9.27501768e-02,
        -5.37407696e-02,  1.32184938e-01,  8.10448602e-02,
         6.96275905e-02,  3.45241986e-02, -3.45069170e-02,
        -8.24541599e-02, -1.30007956e-02,  5.49336709e-02,
         8.09830427e-02,  9.75692272e-02,  1.33590894e-02,
        -1.02557443e-01,  1.09500021e-01, -1.02464393e-01,
        -8.63782167e-02,  1.96165651e-01,  9.97928437e-03,
         1.34713855e-02,  7.86842629e-02,  5.81782982e-02,
        -4.74950634e-02, -7.36115873e-02, -2.86515281e-02,
        -3.21474671e-03,  1.35241505e-02,  2.07400415e-02,
        -2.81391088e-02, -5.59066832e-02, -1.63930133e-02,
         5.76668084e-02, -7.34250247e-02,  6.56741410e-02,
         6.78066211e-03,  2.17143476e-01, -1.77916348e-01,
        -1.66024528e-02,  7.67319556e-03,  1.33220896e-01,
        -9.18806568e-02,  2.86759040e-03, -1.66994482e-01,
         1.15551919e-01,  2.51331553e-02,  3.63404155e-02,
        -1.94187313e-02,  1.22499630e-01, -4.45030928e-0